In [1]:
import torch
import numpy as np
from skimage import io, color, transform
from torch.distributions import MultivariateNormal

def generate_Y_from_image_gp(z, caseno,
                              noise_std=0.1,
                              kernel_var=1.0,
                              lengthscale=1.0):
    """
    用 GP 而非独立噪声生成 Y：
    - 均值函数在边界内为 5，外为 -5
    - 核使用 RBF(k(z,z')=kernel_var*exp(-0.5*||z-z'||^2/lengthscale^2))
    - 观测噪声为 noise_std^2
    """
    # —— 1. 读取并二值化边界图像 —— 
    image_file = {4: 'bound3.png',
                  5: 'bound1.png',
                  6: 'bound4.png'}[caseno]
    I = io.imread(image_file)
    bw = color.rgb2gray(I) > 0.5
    G = 100
    bw = transform.resize(bw, (G, G), anti_aliasing=False) > 0.5
    gx = np.linspace(-3, 3, G)
    gy = np.linspace(-3, 3, G)

    # —— 2. 先把 z 转 numpy，用来判断每个 z 是否在内部 —— 
    z_np = z.cpu().numpy()
    inside_mask = []
    for (z0, z1) in z_np:
        ix = np.argmin(np.abs(gx - z0))
        iy = np.argmin(np.abs(gy - z1))
        inside_mask.append(bw[iy, ix])
    inside_mask = torch.tensor(inside_mask, device=z.device)

    # —— 3. 构造 GP 的均值向量 m —— 
    #    内部点均值  5，外部点均值 -5
    m = torch.where(inside_mask, 5.0, -5.0)

    # —— 4. 构造 RBF 核矩阵 K —— 
    #    K_ij = kernel_var * exp(-0.5 * ||z_i - z_j||^2 / lengthscale^2)
    #    + noise_std^2 对角观测噪声
    N = z.shape[0]
    # 计算 pairwise squared distances
    diff = z.unsqueeze(1) - z.unsqueeze(0)              # [N,N,2]
    dist2 = (diff**2).sum(-1)                          # [N,N]
    K = kernel_var * torch.exp(-0.5 * dist2 / lengthscale**2)
    K += (noise_std**2) * torch.eye(N, device=z.device)

    # —— 5. 从 MultivariateNormal(m, K) 中一次性采样 —— 
    mvn = MultivariateNormal(m, covariance_matrix=K)
    Y = mvn.sample()   # 形状 [N]

    return Y


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, color, transform

def generate_Y_with_numpy(z_np, caseno,
                          noise_std=0.0,
                          kernel_var=1.0,
                          lengthscale=1.0):
    """
    用纯 NumPy 实现的 GP 生成 Y。
    """
    # 读取并二值化
    image_file = {4: 'bound3.png',
                  5: 'bound1.png',
                  6: 'bound4.png'}[caseno]
    I = io.imread(image_file)
    bw = color.rgb2gray(I) > 0.5
    G = 100
    bw = transform.resize(bw, (G, G), anti_aliasing=False) > 0.5
    gx = np.linspace(-3, 3, G)
    gy = np.linspace(-3, 3, G)
    
    # 判断内部外部
    inside = np.array([
        bw[np.argmin(np.abs(gy - y)), np.argmin(np.abs(gx - x))]
        for x, y in z_np
    ])
    m = np.where(inside, 5.0, -5.0)
    
    # 构造核矩阵
    diff = z_np[:, None, :] - z_np[None, :, :]
    dist2 = np.sum(diff**2, axis=-1)
    K = kernel_var * np.exp(-0.5 * dist2 / lengthscale**2)
    K += noise_std**2 * np.eye(len(z_np))
    
    # GP 采样
    Y = np.random.multivariate_normal(m, K)
    return Y

# 构造网格和 z
G_plot = 100
gx = np.linspace(-3, 3, G_plot)
gy = np.linspace(-3, 3, G_plot)
xx, yy = np.meshgrid(gx, gy)
z_np = np.vstack([xx.ravel(), yy.ravel()]).T

# 生成 Y
Y_vals = generate_Y_with_numpy(z_np, caseno=5, noise_std=0.0)

# 可视化
plt.figure()
plt.scatter(z_np[:, 0], z_np[:, 1], c=Y_vals, s=5)
plt.colorbar(label='Y 值')
plt.xlabel('z0')
plt.ylabel('z1')
plt.title('基于图像边界的 Y 值分布 (NumPy GP)')
plt.show()
